# Step 9 - Supplements
## Project: Growing Urban Bicycle Networks

This notebook supplements some calculations.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2021-02-15  
Last modified: 2021-02-15

## Preliminaries

### Parameters

In [ ]:
debug = True # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Calculate efficiencies on the simplified routed graphs

In [ ]:
pois = ["grid", "railwaystation"]
measures = ["betweenness", "closeness", "random"]
combs = list(itertools.product(pois, measures))
# combs = [combs[4]]
pp.pprint(combs)


In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Analyzing results")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
        
    for c in combs:
        poi_source = c[0]
        prune_measure = c[1]

        # Load POIs
        with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
            nnids = [int(line.rstrip()) for line in f]

        # Load results
        filename = placeid + '_poi_' + poi_source + "_" + prune_measure
        resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
        res = pickle.load(resultfile)
        resultfile.close()

        # Calculate
        # output contains lists for all the prune_quantile values of the corresponding results
        output, covs = calculate_metrics_additively(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk = buffer_walk, numnodepairs = numnodepairs, verbose = False, return_cov = True, Gexisting = {}, output = {"efficiency_global_routed": [], "efficiency_local_routed": []})

        # Read old results
        filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".csv"
        results_old = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names = True)

        # Stitch the results together
        output_final = {}
        for fieldname in results_old.dtype.names:
            if fieldname != "efficiency_global_routed" and fieldname != "efficiency_local_routed":
                output_final[fieldname] = list(results_old[fieldname])
        for fieldname in list(output.keys()):
            output_final[fieldname] = output[fieldname]

        # Overwrite old stuff
        write_result(output_final, "dict", placeid, poi_source, prune_measure, ".csv")

    #     # Testing the simplification
    #     fig = initplot()
    #     map_center = nxdraw(G_carall, "carall")
    #     nxdraw(G_simplified, "abstract", map_center, weighted = 6)

In [ ]:
# # Testing stuff
# # print(ig.Graph.from_networkx(ox.simplify_graph(nx.MultiDiGraph(res['GTs'][1].to_networkx()))))
# G = res['GTs'][1]
# print(G)
# G_temp = copy.deepcopy(G)
# G_temp.es["length"] = G_temp.es["weight"]
# output = ig.Graph.from_networkx(ox.simplify_graph(nx.MultiDiGraph(G_temp.to_networkx())).to_undirected())
# print(output)
# output.es["weight"] = output.es["length"]

# # for e in res['GTs'][2].es:
# #     print(e)